Code from previous exercise

In [1]:
from __future__ import division, unicode_literals 
import pandas as pd
import numpy as np

import requests #to read HTML pages

! pip install BeautifulSoup4 

from bs4 import BeautifulSoup

#url of wikipedia
website_url = requests.get('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#read the right table
soup = BeautifulSoup(website_url,'lxml')
wikitable=soup.find('table',{'class':'wikitable sortable'})

#get the table column atributes
header=[th.text.rstrip() for th in wikitable.find_all('th')]

#get the data from the table
torontorawdata=[td.text.rstrip() for td in wikitable.find_all('td')]

#transform the data into lists
PostalCode=[]
Borough=[]
Neighbourhood=[]

lengthoftorontorawdata=len(torontorawdata)

i=0
while i < lengthoftorontorawdata:
    PostalCode.append(torontorawdata[i])
    Borough.append(torontorawdata[i+1])
    Neighbourhood.append(torontorawdata[i+2])
    i=i+3

#build matrix with lists
rawdata=np.array([PostalCode,Borough,Neighbourhood])
rawdata=rawdata.transpose()


#build a df were we will store all the list data
torontodata=pd.DataFrame(rawdata, columns=[header[0],header[1],header[2]])

#remove all the cases in which there is no Borough assigned
NoBoroughIndex=torontodata[torontodata['Borough']=='Not assigned'].index
torontodata.drop(NoBoroughIndex , inplace=True)

#for all those cases in which the Neighbourhood is 'Not assigned' we will copy the name of the Borough
NoNeighbourIndex=torontodata[torontodata['Neighbourhood']=='Not assigned'].index

lengthofnoneighbour=len(NoNeighbourIndex)

i=0
while i < lengthofnoneighbour:
    torontodata.at[NoNeighbourIndex[i],'Neighbourhood']=torontodata.at[NoNeighbourIndex[i],'Borough']
    i=i+1
   

# group by postcode

cleantorontodata=torontodata.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(','.join).reset_index()

This is the table

In [2]:
cleantorontodata

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [8]:
#read the data from the CSV

coordinates = pd.read_csv('http://cocl.us/Geospatial_data')

In [14]:
#create two lists that we will add to the dataframe
Latitude=[]
Longitude=[]

#sort the CSV file by Postal Code
coordinatesorted=coordinates.sort_values(by="Postal Code")
Latitude=coordinatesorted['Latitude']
Longitude=coordinatesorted['Longitude']

#sort the Torontodata by postal code
cleantorontodatasorted=cleantorontodata.sort_values(by="Postal Code")

#add the columns latitude and longitude to the cleantorontodatasorted dataframe
cleantorontodatasorted['Latitude']=Latitude
cleantorontodatasorted['Longitude']=Longitude


Transformed dataframe

In [15]:
cleantorontodatasorted

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
